In [1]:
import functools

import numpy as np
import tensorflow as tf

In [2]:
train_data = ['banglish.csv']

In [8]:
import csv

filename = 'banglish.csv'
raw_data = open(filename, 'rt')
reader = csv.reader(raw_data, delimiter=',', quoting=csv.QUOTE_NONE)
x = list(reader)
train_data = np.array(x).astype('str')
print(train_data[1])


[' Allah Uni to amr hall r apu ' '0']


In [9]:
text = []
label = []
i=0
for x in train_data:
    text.append(x[0])
    label.append(x[1])

In [10]:
print([text],' ',[label])

[['Data', ' Allah Uni to amr hall r apu ', ' Amader ektai dabi amader desher moto niriho Islam prio manush jeno hoyranir shikar na hoy.... Mr. Erdogan e bepare shocheton asen bole mone kori..... ', ' Aj ki kono mach asay ', ' bjibike fair er wodar dey na ken. ', ' amra tho bograi taki tahole dakay oto taka koros kore jabo ki kore ', ' muslim hisabe amader uchit oder asroy deya... dut amra ki asolei muslim rasthe baskokri..?????????????????? ', ' Train e cole aste bolenn ', ' hahaha bolar kichu nai sob gayni vai der comment ai khane ', ' movie na jene r kotha na bola e uchit.alia er hero onno 3 actor. ', ' jr jr icha.......... r tara bujha e korcha ti nah ata neya amra matha gamanor ki dorkr... abr adr movie chara amra toh cholta pari nah tila kn ato base korbo ', ' filmeto sobkicui hoi r atate age ki jai ase ', ' hi sob ke. a deya amra ke korbo ', ' amer je onk kisu churi holo ta ke uddar karbe ', ' Hahhaha ki je bolen dada ', ' dukhu kostu niye amader jibon... ma go apnk salam...duia 

In [13]:
# text cleaning.......
import re
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

def clean_text(df):
    all_reviews = []
    #lines = df["Review_text"].values.tolist()
    for text in df:
        text = text.lower()
        
        pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
        text = pattern.sub('', text)
        
        emoji = re.compile("["
                           u"\U0001F600-\U0001FFFF"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
        text = emoji.sub(r'', text)
        
        text = re.sub(r"i'm", "i am", text)
        text = re.sub(r"he's", "he is", text)
        text = re.sub(r"she's", "she is", text)
        text = re.sub(r"that's", "that is", text)        
        text = re.sub(r"what's", "what is", text)
        text = re.sub(r"where's", "where is", text) 
        text = re.sub(r"\'ll", " will", text)  
        text = re.sub(r"\'ve", " have", text)  
        text = re.sub(r"\'re", " are", text)
        text = re.sub(r"\'d", " would", text)
        text = re.sub(r"\'ve", " have", text)
        text = re.sub(r"won't", "will not", text)
        text = re.sub(r"don't", "do not", text)
        text = re.sub(r"did't", "did not", text)
        text = re.sub(r"can't", "can not", text)
        text = re.sub(r"it's", "it is", text)
        text = re.sub(r"couldn't", "could not", text)
        text = re.sub(r"have't", "have not", text)
        
        text = re.sub(r"[,.\"!@#$%^&*(){}?/;`~:<>+=-]", "", text)
        tokens = word_tokenize(text)
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        words = [word for word in stripped if word.isalpha()]
#         stop_words = set(stopwords.words("english"))
#         stop_words.discard("not")
#         words = [w for w in words if not w in stop_words]
        words = ' '.join(words)
        all_reviews.append(words)
    return all_reviews

text = clean_text(text)
text[0:20]

['data',
 'allah uni to amr hall r apu',
 'amader ektai dabi amader desher moto niriho islam prio manush jeno hoyranir shikar na hoy mr erdogan e bepare shocheton asen bole mone kori',
 'aj ki kono mach asay',
 'bjibike fair er wodar dey na ken',
 'amra tho bograi taki tahole dakay oto taka koros kore jabo ki kore',
 'muslim hisabe amader uchit oder asroy deya dut amra ki asolei muslim rasthe baskokri',
 'train e cole aste bolenn',
 'hahaha bolar kichu nai sob gayni vai der comment ai khane',
 'movie na jene r kotha na bola e uchitalia er hero onno actor',
 'jr jr icha r tara bujha e korcha ti nah ata neya amra matha gamanor ki dorkr abr adr movie chara amra toh cholta pari nah tila kn ato base korbo',
 'filmeto sobkicui hoi r atate age ki jai ase',
 'hi sob ke a deya amra ke korbo',
 'amer je onk kisu churi holo ta ke uddar karbe',
 'hahhaha ki je bolen dada',
 'dukhu kostu niye amader jibon ma go apnk salamduia kori allah apnr kosto dur kore den',
 'vi tomar kotha suna mona holo tumi

In [14]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import gensim, re
import pandas as pd


features = 500
tokenizer = Tokenizer(num_words = features)

# আমাদের টোকেনাইজারকে পুরো টেক্সটে ফিট করতে হবে 
tokenizer.fit_on_texts(text)

# আমাদের টোকেনাইজার চেনে সেরকম শব্দ নিয়ে আসতে হবে 
word_index = tokenizer.word_index

# ম্যাট্রিক্স এর মধ্যে টোকেনকে ফেলি 
X = tokenizer.texts_to_sequences(text)
X = pad_sequences(X,maxlen=15)

# লেবেল তৈরি করি 
y = np.asarray(pd.get_dummies(label))


In [15]:
print(X[2145:2158])

[[  0   0   0   0   0   0   0   0   0   0 336  32 138   5  14]
 [  0   0   0   0   0   0   0   0   0   8  32  70  19 386  22]
 [  0   0   0   0   0   0   0   0   0   0  69 224  22 263  24]
 [  0  24  93  70 436   5  31  14  57  62 150  93  70 147 218]
 [  0   0   0   0   0   0  58   6 436 179 430  26  32 179 442]
 [ 48 436 163  44  40 333 186 340   1 163 100  63  24  93 147]
 [  0   0   0   0   0   0   0   0  32  48  17 224   7  22 493]
 [  0   0   0   0   0   0   0   0   0   0   8 198  41  32  80]
 [453   7   6  76  93 100  59  31  14 257 150  24  93  70  57]
 [  0   0   0   0   0   0   0   0   0   0   0  93  70 147 218]
 [  0   0   0   0   0   0   0   0   0   1 192 438  93  70 102]
 [  0   0 315  28   1 150  32 222 224 222  70  36 257 198 222]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0  16  11]]


In [16]:
pd.get_dummies(label)

,,-1,0,1,Label
0,0,0,0,0,1
1,0,0,1,0,0
2,0,0,1,0,0
3,0,0,1,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,1,0,0
7,0,0,1,0,0
8,0,0,1,0,0
9,0,0,1,0,0


In [17]:
#pd.get_dummies(label[:2000])
#pd.get_dummies(label[2001:4000])
#pd.get_dummies(label[4000:])


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=True)

y_train.shape

(5402, 5)

In [19]:
lines_from_file = []

for line in text:
    lines_from_file.append(line)
      
tokenized_lines = []
for single_line in lines_from_file:
    tokenized_lines.append(single_line.split())

In [20]:
print(tokenized_lines[1050:1068])

[['dekhe', 'mone', 'hoy', 'muslimchehara', 'sundor'], ['avabei', 'ekdin', 'mrittu', 'asbe', 'apnar', 'duyare', 'sedin', 'ar', 'keu', 'woddhar', 'korte', 'parbe', 'na', 'apnake', 'sob', 'opokormer', 'hisab', 'dite', 'hobe'], ['kitnu', 'allah', 'ar', 'jedin', 'atka', 'porbe', 'sedin', 'karo', 'khomota', 'nai', 'dorja', 'bangar'], ['onar', 'helth', 'check', 'korar', 'jonno', 'singapor', 'pathano', 'hok'], ['bahir', 'korar', 'ki', 'dorkar', 'silo'], ['kintu', 'jara', 'manonio', 'montrir', 'ai'], ['toilet', 'a', 'hole', 'valo', 'hoto'], ['assalamo', 'alikum', 'janab', 'asha', 'kori', 'rajnity', 'taa', 'chapa', 'na', 'pari', 'shob', 'somoy', 'shotty', 'kotha', 'bolben', 'allah', 'onak', 'taka', 'poysha', 'diasen'], ['oh', 'no', 'arek', 'ta', 'miss'], ['spinner', 'onek', 'ase', 'doler', 'vitor', 'pacer', 'holei', 'valo', 'hoi', 'shofiuler', 'bodli', 'alamin', 'ke', 'neoa', 'jete', 'pare'], ['taizulbut', 'amr', 'mone', 'hy', 'kal', 'mosarof', 'hossen', 'rubel', 'k', 'safiul', 'er', 'replacemen

In [22]:
word_model = gensim.models.Word2Vec(tokenized_lines, size=400, min_count=1, iter=10)
#word_model = gensim.models.Word2Vec(text, size=300, min_count=1, iter=10)

In [23]:
word_model.wv.most_similar(positive='jai')

[('jay', 0.9999862909317017),
 ('ba', 0.9999842643737793),
 ('j', 0.9999833106994629),
 ('keno', 0.9999815821647644),
 ('diye', 0.9999805092811584),
 ('kno', 0.9999804496765137),
 ('jabe', 0.9999804496765137),
 ('korbe', 0.9999799728393555),
 ('abar', 0.9999798536300659),
 ('manus', 0.9999798536300659)]

In [24]:
# নতুন ম্যাট্রিক্সে সেভ করি ভেক্টরগুলোকে 
embedding_matrix = np.zeros((len(word_model.wv.vocab) + 1, 300))
for i, vec in enumerate(word_model.wv.vectors):
    embedding_matrix[i] = vec

ValueError: could not broadcast input array from shape (400) into shape (300)

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding

# লেয়ারকে ইনিশিয়ালাইজ করি, অবশ্যই সিকোয়েন্সিয়াল মডেল  
model = Sequential()
# শব্দ ভেক্টরকে এমবেড করি 
model.add(Embedding(len(word_model.wv.vocab)+1, 300, input_length=X.shape[1], weights=[embedding_matrix], trainable=True))
# ডেটার মধ্যে কো-রিলেশন বোঝার চেষ্টা করতে দেই মডেলকে 
model.add(LSTM(1000,return_sequences=False))
model.add(Dense(y.shape[1],activation="softmax"))
# মডেলের কাঠামো দেখি 
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 15, 300)           4064100   
_________________________________________________________________
lstm (LSTM)                  (None, 1000)              5204000   
_________________________________________________________________
dense (Dense)                (None, 5)                 5005      
Total params: 9,273,105
Trainable params: 9,273,105
Non-trainable params: 0
_________________________________________________________________


In [26]:
# মডেলকে কম্পাইল করি 

model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['acc'])

In [27]:
# মডেলকে ট্রেইন করি ব্যাচ দিয়ে, দেখানোর জন্য

batch = 32
epochs = 6
model.fit(X_train, y_train, batch, epochs)

Epoch 1/6
169/169 [==============================] - 103s 608ms/step - loss: 1.0703 - acc: 0.4485
Epoch 2/6
169/169 [==============================] - 104s 617ms/step - loss: 0.9287 - acc: 0.5424
Epoch 3/6
169/169 [==============================] - 103s 608ms/step - loss: 0.8892 - acc: 0.5733
Epoch 4/6
169/169 [==============================] - 101s 595ms/step - loss: 0.8486 - acc: 0.5983
Epoch 5/6
169/169 [==============================] - 100s 592ms/step - loss: 0.8223 - acc: 0.6155
Epoch 6/6
169/169 [==============================] - 98s 579ms/step - loss: 0.7969 - acc: 0.6366


In [28]:
model.evaluate(X_test, y_test)

19/19 [==============================] - 2s 91ms/step - loss: 0.9739 - acc: 0.5391


[0.9738628268241882, 0.5391014814376831]

In [33]:
sents = ['valo lage na', 'amr khub kharap lagce', 'onek sundor', 'ami ar ki bolbo somay asla bolbo', 'allah k tumi','Allah Uni to amr hall r apu', 'khub kharap hoiche']
# sent_n = [[word_index[w]+3 for w in s.split()] for s in sents]
sent_n = tokenizer.texts_to_sequences(sents)
X_new = pad_sequences(sent_n,maxlen=15)
prediction = model.predict(X_new)

In [34]:
# '১' মানে পজিটিভ, '০' মানে নেগেটিভ, কারণ pd.get_dummy() দেখিয়েছে কলাম '০' মানে নেগেটিভ আর কলাম '১' মানে পজিটিভ  

np.argmax(prediction, axis=1)

array([1, 1, 3, 2, 2, 2, 1], dtype=int64)